In [1]:
pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.9 MB/s eta 0:00:00


In [7]:
import yfinance as yf
import pandas as pd

tickers = ["BTC-USD", "CL=F"]
start_date  = "2024-01-01"
end_date = "2024-12-31"

data = yf.download(tickers, start_date, end=end_date)
display(data.head())

/tmp/ipython-input-860155399.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start_date, end=end_date)
[*********************100%***********************]  2 of 2 completed


Price              Close                     High                      Low  \
Ticker           BTC-USD       CL=F       BTC-USD       CL=F       BTC-USD   
Date                                                                         
2024-01-01  44167.332031        NaN  44175.437500        NaN  42214.976562   
2024-01-02  44957.968750  70.379997  45899.707031  73.639999  44176.949219   
2024-01-03  42848.175781  72.699997  45503.242188  73.230003  40813.535156   
2024-01-04  44179.921875  72.190002  44770.023438  74.000000  42675.175781   
2024-01-05  44162.691406  73.809998  44353.285156  74.239998  42784.718750   

Price                          Open                  Volume            
Ticker           CL=F       BTC-USD       CL=F      BTC-USD      CL=F  
Date                                                                   
2024-01-01        NaN  42280.234375        NaN  18426978443       NaN  
2024-01-02  70.059998  44187.140625  71.709999  39335274536  330992.0  
2024-01-03  69.279999  44961.601562  70.500000  46342323118  334861.0  
2024-01-04  71.059998  42855.816406  73.040001  30448091210  344467.0  
2024-01-05  72.209999  44192.980469  72.400002  32336029347  325525.0

In [11]:
# Select the 'Close'price and calculate daily returns
close_prices = data['Close']
daily_returns = close_prices.pct_change()*100

daily_returns = daily_returns.dropna()


display(daily_returns.dropna())

/tmp/ipython-input-2010468100.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = close_prices.pct_change()*100


Ticker,BTC-USD,CL=F
Date,,
2024-01-03,-4.692812,3.296391
2024-01-04,3.108058,-0.701506
2024-01-05,-0.039001,2.244071
2024-01-06,-0.392857,0.000000
2024-01-07,-0.104793,0.000000
...,...,...
2024-12-26,-3.528407,-0.684730
2024-12-27,-1.702226,1.407635
2024-12-28,1.060980,0.000000


In [13]:
import statsmodels.api as sm

# Define the dependent and Independent variables
Y = daily_returns ['BTC-USD']
X = daily_returns ['CL=F']

# Add a constant (intercept) to the independant variable
X = sm.add_constant(X)

# Perform OLS regression
model=sm.OLS(Y, X).fit()

# Display the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                BTC-USD   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.057
Date:                Wed, 24 Sep 2025   Prob (F-statistic):              0.152
Time:                        04:16:47   Log-Likelihood:                -887.85
No. Observations:                 363   AIC:                             1780.
Df Residuals:                     361   BIC:                             1787.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2401      0.147      1.634      0.1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Prepare the data
X = daily_returns[['CL=F']].dropna() # Independent variable (CL=F)
Y = daily_returns['BTC-USD'].loc[X.index] # Dependent variable (BTC-USD), align indices

# Split data into training and testing sets (80/20 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test)

# Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
mae = mean_absolute_error(Y_test, Y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")